In [13]:
from glob import glob
import os
import os.path
from PIL import Image
import cv2
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5l')

# Images
for f in ['zidane.jpg', 'bus.jpg']:
    torch.hub.download_url_to_file('https://ultralytics.com/images/' + f, f)  # download 2 images
img1 = Image.open('zidane.jpg')  # PIL image
img0 = Image.open('11.jpg')
img2 = cv2.imread('bus.jpg')[..., ::-1]  # OpenCV image (BGR to RGB)
img4 = Image.open('/data_disk_10TB/office_web_images/6133.jpg')
imgs = [img1, img0, img2, img4]  # batch of images

# Inference
results = model(imgs)  # includes NMS
# Results
# results.print()  
results.show() # or .show()

results.xyxy[0]  # img1 predictions (tensor)
results.pandas().xyxy[3]  # img1 predictions (pandas)
#      xmin    ymin    xmax   ymax  confidence  class    name
# 0  749.50   43.50  1148.0  704.5    0.874023      0  person
# 1  433.50  433.50   517.5  714.5    0.687988     27     tie
# 2  114.75  195.75  1095.0  708.0    0.624512      0  person
# 3  986.00  304.00  1028.0  420.0    0.286865     27     tie

Using cache found in /home/maxim/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-12-14 torch 1.8.0 CUDA:0 (TITAN RTX, 24217MiB)

Fusing layers... 
Model Summary: 367 layers, 46533693 parameters, 0 gradients
Adding AutoShape... 


,xmin,ymin,xmax,ymax,confidence,class,name
0,455.170074,6.091631,1080.000000,646.452576,0.919255,63,laptop
1,1.231018,490.013275,563.368652,634.279175,0.696675,63,laptop
2,41.530380,2.331192,733.524414,510.334686,0.670114,0,person
3,60.388454,134.259598,230.648956,331.347443,0.652740,67,cell phone


In [14]:
import time
imgs = glob('/data_disk_10TB/office_web_images/*.jpg')

In [15]:
def is_person_in_values(xyxy, i):
    return 'person' in xyxy[i]['name'].values

def get_tgt_path(filepath):
    to_replace = filepath.split('/')
    to_replace[2] = 'office_web_images_no_humans'
    return '/'.join(to_replace)

def move_to_folder(source, target):
    pass

range_i = list(range(len(imgs)))
for i in range(0, len(imgs), 5):
    process_i = range_i[i:i+5]
    images = []
    img_names = []
    tgt_names = []
    for i_curr in process_i:
        img_name = imgs[i_curr]
#         print(img_name)
        tgt_path = get_tgt_path(img_name)
        try:
            im = Image.open(img_name)
            images.append(im)
            img_names.append(img_name)
            tgt_names.append(tgt_path)
        except:
            if os.path.exists(img_name):
                os.remove(img_name)
            
        
#     images = [img1, img0, img2, img4]  # batch of images
    if len(images) > 0:
        results = model(images)
        xyxy = results.pandas().xyxy  # img1 predictions (pandas)
        for j in range(len(images)):
            if is_person_in_values(xyxy, j):
                os.rename(img_names[j], tgt_names[j])
        time.sleep(0.01)

In [42]:
is_person_in_values(results.pandas().xyxy, 3)

True

In [ ]:
import torch

In [3]:
torch.cuda

<module 'torch.cuda' from '/home/maxim/anaconda3/envs/yolov5/lib/python3.7/site-packages/torch/cuda/__init__.py'>

In [ ]:
torch.__version__()